In [ ]:
import torch
torch.cuda.is_available()


True

If this returns True, you're good to go! If not, go to Runtime > Change runtime type > GPU.

In [ ]:
!pip install -U -q torch transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Or float32 if no GPU/float16 support
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
for param in model.parameters():
    param.requires_grad = False

for name, param in model.model.layers[27].named_parameters():
    if torch.is_floating_point(param):
        param.requires_grad = True

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.0,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "model.layers.27.self_attn.q_proj",
        "model.layers.27.self_attn.k_proj",
        "model.layers.27.self_attn.v_proj",
        "model.layers.27.self_attn.o_proj",
    ],
)

model = get_peft_model(model, lora_config)

# model.print_trainable_parameters()

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")

Trainable: base_model.model.model.layers.27.self_attn.q_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.27.self_attn.q_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.27.self_attn.k_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.27.self_attn.k_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.27.self_attn.v_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.27.self_attn.v_proj.lora_B.default.weight
Trainable: base_model.model.model.layers.27.self_attn.o_proj.lora_A.default.weight
Trainable: base_model.model.model.layers.27.self_attn.o_proj.lora_B.default.weight


In [ ]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

df = pd.read_csv("japan_noto.csv")  
df = df.dropna(subset=['gpe'])
def build_prompt(row):
    return f"### Question:\n Identify locations like country, city, or any local areas in the following sentence and write them in a comma-separated manner (no additional details required): {row['text']}\n\n### Answer:\n{row['gpe']}"

df["sentence"] = df.apply(build_prompt, axis=1)

# Create HuggingFace Dataset
dataset = Dataset.from_pandas(df[["sentence"]])

# Optional: train/test split
dataset = dataset.train_test_split(test_size=0.1, seed=42)

In [ ]:
def tokenize(example):
    tokens = tokenizer(
        example["sentence"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_datasets = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/592 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=10,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
)


In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(590))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(20))

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset.remove_columns(["sentence"]),  # Remove raw text column
    eval_dataset=small_test_dataset.remove_columns(["sentence"]),
)


In [ ]:
torch.cuda.empty_cache()

In [ ]:
print("🚀 Starting Fine-Tuning...")
trainer.train()

🚀 Starting Fine-Tuning...


Step,Training Loss


TrainOutput(global_step=140, training_loss=1.1983021327427454, metrics={'train_runtime': 695.7461, 'train_samples_per_second': 8.48, 'train_steps_per_second': 0.201, 'total_flos': 2.613133708886016e+16, 'train_loss': 1.1983021327427454, 'epoch': 9.337837837837839})

In [ ]:
import re

In [ ]:
def generate_prediction(review_text):
    inputs = tokenizer(review_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=2000, pad_token_id=tokenizer.eos_token_id)
    content = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return re.sub(r'^.*?</think>\s*', '', content, flags=re.DOTALL)

# Example reviews


# Dedicated prompt engineering required
reviews = ["Identify locations like country, city, or any local areas in the following sentence and write them in a comma-separated manner (no additional details required): Earthquake in Japan detected noto peninsula people need help"]

# Run predictions
for review in reviews:
    print(f"Q: {review}")
    print(f"A: {generate_prediction(review)}")


Q: Identify locations like country, city, or any local areas in the following sentence and write them in a comma-separated manner (no additional details required): Earthquake in Japan detected noto peninsula people need help
A: Identify locations like country, city, or any local areas in the following sentence and write them in a comma-separated manner (no additional details required): Earthquake in Japan detected noto peninsula people need help in Japan


In [ ]:
import csv
import pandas as pd

In [ ]:
csv_file = "japan_noto.csv"

# Read the existing CSV file
df = pd.read_csv(csv_file)
df = df[:50]
# Add new columns if not present
if "finetuned_26_27" not in df.columns:
    df["finetuned_26_27"] = ""

# Process each row and update detected_gpe and response_time
for index, row in df.iterrows():
    if pd.isna(row["finetuned_26_27"]) or row["finetuned_26_27"].strip() == "":
        text = row["text"]

        # Add instruction with a newline before the actual text
        prompt = f"Identify locations like country, city, or any local areas in the following sentence and write them in a comma-separated manner (no additional details required):\n\n{text}"

        detected_gpe = generate_prediction(prompt)  # Get LLM response

        # Clean multiline response: strip spaces, join with commas if needed
        detected_gpe = ", ".join(line.strip() for line in detected_gpe.split("\n") if line.strip())

        # Update DataFrame
        df.at[index, "finetuned_26_27"] = detected_gpe

        # Save the updated row immediately to keep tracking progress
        df.to_csv(csv_file, index=False, quoting=csv.QUOTE_NONNUMERIC)

print("CSV file updated with detected GPE values and response times!")

CSV file updated with detected GPE values and response times!


In [ ]:
from google.colab import files
files.download("japan_noto.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_pretrained("./finetuned_model")

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# # Load the LoRA weights
# model = PeftModel.from_pretrained(model, "./finetuned_model")
